# Model definition

In [1]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models


model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(100, 100, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))

model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(43, activation='softmax'))

model.summary()

/opt/anaconda3/envs/MPS-Torch/lib/python3.12/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
2025-05-16 21:07:20.024942: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M3 Pro
2025-05-16 21:07:20.024970: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 36.00 GB
2025-05-16 21:07:20.024975: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 13.50 GB
I0000 00:00:1747393640.025158 22939543 pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
I0000 00:00:1747393640.025584 22939543 pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/devi

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 98, 98, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 49, 49, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 47, 47, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 23, 23, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 21, 21, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 10, 10, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 8, 8, 64)       │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 4, 4, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 2, 2, 64)       │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │        65,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 43)             │         5,547 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 234,411 (915.67 KB)

 Trainable params: 234,411 (915.67 KB)

 Non-trainable params: 0 (0.00 B)

# Do some shit around the model

In [2]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [3]:
import os
import numpy

# read in train data
TRAINING_DATA_PATH = './2025_A2/train'
TRAIN_LABEL_PATH = TRAINING_DATA_PATH + '/train_metadata.csv'

def get_labels(labels_path):
    # load labels as dict 
    labels = {}
    mxm_labels = 0
    with open(labels_path, 'r') as f:
        counter:int = 0
        for line in f:
            # skip the first line
            if counter == 0:
                counter += 1
                continue
            line = line.strip().split(',')
            # line [1] is the image name, line [2] is the label
            labels[line[1]] = int(line[2])
            if int(line[2]) > mxm_labels:
                mxm_labels = int(line[2])
    print(f"Max label: {mxm_labels}")
    return labels

def load_and_preprocess_images(image_folder, labels_dict, target_size=(100, 100)):
    images = []
    labels = []
    
    for filename in os.listdir(image_folder):
        if filename.endswith(".jpg"): 
            img_path = os.path.join(image_folder, filename)
            
            # Read image with TensorFlow
            img = tf.io.read_file(img_path)  # Read the image file
            img = tf.image.decode_jpeg(img, channels=3)  # Decode the JPEG image (for JPG files)

            # Resize the image to the target size (100x100)
            img_resized = tf.image.resize(img, target_size)
            
            # Normalize image to [0, 1]
            img_normalized = img_resized / 255.0
            images.append(img_normalized)
            
            # Get the label from the dictionary
            label = labels_dict.get(filename)
            labels.append(label)
    
    # Convert lists to numpy arrays
    images = numpy.array(images)
    labels = numpy.array(labels)
    
    return images, labels

overall_images, overall_labels = load_and_preprocess_images(TRAINING_DATA_PATH, get_labels(TRAIN_LABEL_PATH))

train_images, test_images = tf.keras.utils.split_dataset(
    overall_images, left_size=0.9)

train_labels, test_labels = tf.keras.utils.split_dataset(
    overall_labels, left_size=0.9)

print(f"Loaded {len(train_images)} images and {len(train_labels)} labels.")
# print(f"Image shape: {train_images[0].shape}")

Max label: 42
Loaded 4939 images and 4939 labels.


In [4]:
# # debug stuffs
# class PrintShapesCallback(tf.keras.callbacks.Callback):
#     def on_batch_end(self, batch, logs=None):
#         # Print target shape and output shape at the end of each batch
#         print(f"Batch {batch + 1}: Target Shape = {logs['y_true'].shape}, Output Shape = {logs['y_pred'].shape}")

In [5]:
# trin the model
model.fit(numpy.array(list(train_images)), numpy.array(list(train_labels)), epochs=10, batch_size=256)

2025-05-16 21:07:53.131286: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2025-05-16 21:07:53.330170: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


Epoch 1/10


/opt/anaconda3/envs/MPS-Torch/lib/python3.12/site-packages/keras/src/backend/tensorflow/nn.py:708: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(
2025-05-16 21:07:53.914839: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


20/20 ━━━━━━━━━━━━━━━━━━━━ 3s 84ms/step - accuracy: 0.0598 - loss: 3.6529
Epoch 2/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 2s 75ms/step - accuracy: 0.1685 - loss: 3.3045
Epoch 3/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 2s 75ms/step - accuracy: 0.2847 - loss: 2.7033
Epoch 4/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 2s 76ms/step - accuracy: 0.3592 - loss: 2.3657
Epoch 5/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 2s 81ms/step - accuracy: 0.3797 - loss: 2.0800
Epoch 6/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 2s 78ms/step - accuracy: 0.4234 - loss: 1.8852
Epoch 7/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 2s 76ms/step - accuracy: 0.4580 - loss: 1.7569
Epoch 8/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 2s 77ms/step - accuracy: 0.5251 - loss: 1.5519
Epoch 9/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 2s 75ms/step - accuracy: 0.5596 - loss: 1.5479
Epoch 10/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 2s 76ms/step - accuracy: 0.6056 - loss: 1.3634


# Model Benchmark

In [6]:
# TEST_DATA_PATH = './2025_A2/test'
# TEST_LABEL_PATH = TEST_DATA_PATH + '/test_metadata.csv'

# test_data, test_labels = load_and_preprocess_images(TEST_DATA_PATH, get_labels(TEST_LABEL_PATH))
# print(f"Loaded {len(test_data)} test images and {len(test_labels)} test labels.")
# print(f"Test image shape: {test_data[0].shape}")

# Evaluate the model on the test data
validation_loss, validation_accuracy = model.evaluate(numpy.array(list(test_images)), numpy.array(list(test_labels)), batch_size=32)

print(f"Validation Loss: {validation_loss}")
print(f"Validation Accuracy: {validation_accuracy}")

 1/18 ━━━━━━━━━━━━━━━━━━━━ 2s 134ms/step - accuracy: 0.5938 - loss: 1.4502

2025-05-16 21:08:10.714309: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.6508 - loss: 1.3027
Validation Loss: 1.2897312641143799
Validation Accuracy: 0.6593806743621826
